In [5]:
!pip3 install ultralytics

In [6]:
import torch

torch.backends.mps.is_available()

True

In [7]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [8]:
from ultralytics import YOLO
#nano
model = YOLO("yolov8n.pt")
model.to('mps')

model.train(data="/Users/tdle26/Documents/mlr/yolo8/data.yaml", epochs=30,device='mps',workers=4*14)

In [9]:
prediction = model.predict('/Users/tdle26/Documents/mlr/yolo8/istockphoto-1278389684-612x612.jpg')
item=prediction[0]
box=item.boxes[0]
#number of item in the photo
len(box)


image 1/1 /Users/tdle26/Documents/mlr/yolo8/istockphoto-1278389684-612x612.jpg: 448x640 7 dogs, 58.7ms
Speed: 2.8ms preprocess, 58.7ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)


1

In [10]:
# Extract the first item from an array of PyTorch Tensor objects
print("Object type:", box.cls[0])
print("Coordinates:", box.xyxy[0])
print("Probability:", box.conf[0])

Object type: tensor(16.)
Coordinates: tensor([130.6427,  97.8394, 250.0875, 321.7167])
Probability: tensor(0.9237)


In [11]:
cords = box.xyxy[0].tolist() #convert to list the coordinates
class_id = box.cls[0].item() #convert to item for tensor with only scalar values
conf = box.conf[0].item()
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

Object type: 16.0
Coordinates: [130.64271545410156, 97.8393783569336, 250.08746337890625, 321.7166748046875]
Probability: 0.9237490892410278


In [12]:
print(item.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [13]:
for box in item.boxes:
  class_id = item.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

Object type: dog
Coordinates: [131, 98, 250, 322]
Probability: 0.92
---
Object type: dog
Coordinates: [245, 32, 364, 368]
Probability: 0.92
---
Object type: dog
Coordinates: [454, 134, 581, 380]
Probability: 0.9
---
Object type: dog
Coordinates: [16, 158, 140, 380]
Probability: 0.88
---
Object type: dog
Coordinates: [343, 83, 473, 366]
Probability: 0.85
---
Object type: dog
Coordinates: [370, 265, 507, 391]
Probability: 0.83
---
Object type: dog
Coordinates: [54, 248, 371, 387]
Probability: 0.76
---


In [14]:
import os
import cv2

image_path = os.path.join('/Users/tdle26/Documents/mlr/yolo8','istockphoto-1278389684-612x612.jpg')
image = cv2.imread(image_path)

for i in range(item.boxes.data.shape[0]):
    cords = item.boxes.xyxy[i].tolist()
    cords = [round(x) for x in cords]
    x,y,w,h = cords[0], cords[1],cords[2],cords[3]
    cv2.rectangle(image, (x,y),(w,h),(0,255,0),2 )
    # Prepare the text to display
    
    predicted_label = item.names[item.boxes.cls[i].item()]
    confidence = round(item.boxes.conf[i].item(), 2)
    label_text = f"Label: {predicted_label}"
    accuracy_text = f"Accuracy: {confidence:.2f}"

    # Draw the text above the bounding box
    cv2.putText(image, label_text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0),thickness=1)
    cv2.putText(image, accuracy_text, (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0),thickness=1)




cv2.imshow("img",image)
cv2.waitKey(1)
cv2.destroyAllWindows()